In [12]:
import pandas as pd

In [11]:
train = pd.read_csv('ann-train.data', header=0, na_values='?')
test = pd.read_csv('ann-test.data', header=0, na_values='?')

print(train.head())
print("------------")
print(test.head())

  0.73 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0.0006 0.015 0.12 0.082 0.146 3  
0  0.24 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0.00025 0.0...                  
1  0.47 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0.0019 0.02...                  
2  0.64 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0.0009 0.01...                  
3  0.23 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0.00025 0.0...                  
4  0.69 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0.00025 0.0...                  
------------
  0.29 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0.0061 0.028 0.111 0.131 0.085 2  
0  0.32 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0.0013 0.01...                   
1  0.35 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0.031 0.2...                   
2  0.21 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0.001 0.018...                   
3  0.22 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0.0004 0.02...                   
4  0.22 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0.0016 0.02...                   
